Business Problem:
----
SmartTech Co. needs to accurately predict laptop prices amidst a diverse market landscape to stay competitive and strategically position its products.

Business Objective: 
----
Developing a robust machine learning model that accurately predicts laptop prices based on various features, enabling SmartTech Co. to maintain competitiveness, understand market positioning, and assess brand influence.

Data Pre-observation:


1,some of the rows in dataset have empty, there is no data in those column, so by applying filters i have deleted those entire rows 

2,there is one column called "Unnamed: 0" so in that there is row numbers starts from '0', its not need in this project so i deleted that column 


3, and there is an unknown column that is also representing row numbers starts from '0' so i deleted that entire column

4,in dataset some of rows have "?" in rows on these 3 columns (inche,memory,weight), so i deleted those 3 rows  

the above all the steps i have done in excel the final data set is shown below 

In [ ]:
import warnings
warnings.filterwarnings("ignore")


In [ ]:
import pandas as pd  # importing pandas library to read dataset and for data manioulation
import numpy as np 
import matplotlib.pyplot as plt # importing pandas library to visualize data
import seaborn as sns 

In [ ]:
#reading the dataset
df = pd.read_csv('final dataset.csv')

In [ ]:
# I am creating multiple copies of data to not disturb the original dataset and use them as and when required.
df1=df.copy()
df2=df.copy()

Data Exploration and Understanding:
----

In [ ]:
df.info()

In [ ]:
# to check the no. of rows and columns
num_rows, num_columns = df.shape
print("shape of dataset",df.shape)
print("size of dataset",df.size)
print("dimensions of dataset",df.ndim)
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

In [ ]:
df.head(5)  

In [ ]:
df.describe(include="all") # to check the statistics of the project

In [ ]:
df.isnull().sum() # to check the number of missing values in each of the variables

No missing values in data set 

In [ ]:
# duplitates remove 
df.drop_duplicates(inplace=True)
print("Number of rows after removing duplicates:", len(df))

In [ ]:
df.info()

No need of finding outliers in this context because:

In the context of laptops, it's common for there to be a wide range of screen sizes ('Inches'), RAM sizes ('Ram'), and weights ('Weight') across different laptop models and manufacturers. These variations are typically considered normal and may not necessarily indicate outliers

here the outliers represent unique high-end or niche laptops, keeping them might help the model capture the price variations associated with those specific features

EDA(Exploratory Data Analysis)
-----

In [ ]:
df['Ram'] = df['Ram'].str.replace('GB','')
df['Weight'] = df['Weight'].str.replace('kg','')

In [ ]:
df['Ram'] = df['Ram'].astype('int32')
df['Weight'] = df['Weight'].astype('float32')

In [ ]:
object_columns = df.select_dtypes(include=['object']).columns.tolist()
numerical_columns=df.select_dtypes(exclude=['object']).columns.tolist()
print("Columns with object data type:", object_columns)
print("Columns with int and float data type:", numerical_columns)

 univerient analsyis
 1. Distribution of numerical variables

In [ ]:
sns.histplot(df['Price'], kde=True)
plt.title('Histogram of Price')
plt.show()

most people buying laptop in between 20000 to 50000

the cost incresing the customers also decreasing

and Price column have left-skewed we have to normalize it 



here the target column is left-skewed or negatively skewed 

so it can lead to biased model predictions

so for that reason i am applying logarithmic transformation (log) 


In [ ]:
sns.histplot(np.log(df['Price']), kde=True)
plt.title('Histogram of Price')
plt.show()

In [ ]:
sns.histplot(df['Inches'], kde=True)
plt.title('Histogram of Inches')
plt.show()

In [ ]:
sns.histplot(df['Ram'], kde=True)
plt.title('Histogram of Ram')
plt.show()

In [ ]:
sns.histplot(df['Weight'], kde=True)
plt.title('Histogram of Weight')
plt.show()

2.Distribution of categorical variables

In [ ]:
df['Company'].value_counts().plot(kind='bar')

most of the people buying Lenovo and Dell companies laptops 

may be the reason is those are on budgets we will confirm this by doing "Bivariate analysis"

In [ ]:
df['TypeName'].value_counts().plot(kind='bar')

most of the people buying Notebook Type laptops 

because these are designed for general-purpose use, including productivity tasks, multimedia consumption, and light gaming

so most of the people who are buying laptop for general purpose

In [ ]:
df['OpSys'].value_counts().plot(kind='bar')

 Bivariate analysis
 1. NUM - NUM

In [ ]:
numerical_columns

In [ ]:
for i in numerical_columns:
    if i=='Price':
        pass
    else:
        correlation = df[i].corr(df['Price'])
        print(f"Correlation between {i} and Price", correlation)
    


only Ram and price have more +ve Correlation compared to other numerical columns

In [ ]:
sns.lineplot(x=df['Ram'],y=df['Price'])

2. CAT-NUM

In [ ]:
print("Cat",object_columns)
print("Num",numerical_columns)

In [ ]:
sns.barplot(x=df['Company'],y=df['Price'],palette="colorblind")
plt.xticks(rotation='vertical')
plt.show()

see most people buying lenovo and dell laptops here we can see their price are below 60000 only 


In [ ]:
sns.barplot(x=df['TypeName'],y=df['Price'],palette="bright")
plt.xticks(rotation='vertical')
plt.show()

Feature Engineering:
----

In [ ]:
df.sample(1)

In [ ]:
#we have multiple data in ScreenResolution like Full HD or not,Touchscreen or not,IPS Panel or not etc..
df['ScreenResolution'].value_counts()

In [ ]:
#1st crating a new column and representing 1 if 'Touchscreen' is present, if not 0

touchscreen_list = []

# Iterate over each value in the 'ScreenResolution' column
for resolution in df['ScreenResolution']:
    # Check if 'Touchscreen' is present in the resolution string
    if 'Touchscreen' in resolution:
        # Append 1 to the list if 'Touchscreen' is present
        touchscreen_list.append(1)
    else:
        # Append 0 to the list if 'Touchscreen' is not present
        touchscreen_list.append(0)

# Creating a new column 'Touchscreen' in the DataFrame and assign the list to it

df['Touchscreen'] = touchscreen_list

In [ ]:
df.sample(1)

lets see is there any relation in between Touchscreen and Price

In [ ]:
sns.countplot(x='Touchscreen',data=df)

most of the laptops didnt have touchscreen

because of highprices 

In [ ]:
sns.barplot(x=df['Touchscreen'],y=df['Price'])

the average price of touchscreen laptops are in between 70000 to 80000

In [ ]:
#next crating a new column and representing 1 if 'IPS' is present, if not 0

#Ips (In-Plane Switching) - IPS panels are known for their superior color accuracy, 
# wide viewing angles, and excellent image quality compared to other panel technologies

IPS_list = []

for resolution in df['ScreenResolution']:
    
    if 'IPS' in resolution:
        IPS_list.append(1)
    else:
        IPS_list.append(0)


df['Ips'] = IPS_list

In [ ]:
df.sample()

In [ ]:
sns.countplot(x='Ips',data=df)

In [ ]:
sns.barplot(x=df['Ips'],y=df['Price'])

in ScreenResolution we have horizontal and vertical resolution of the display in pixels by using this we can calculate Pixels Per Inch (PPI)

Laptops with higher PPI displays may be positioned as premium products compared to those with lower PPI displays

PPI formula : you need to know the horizontal and vertical resolution of the display in pixels, as well as the diagonal size of the display in inches.

PPI= ((horizontal resolution)**2 + (vertical resolution)**2)**0.5/diagonal size(inches)

 


In [ ]:
new = df['ScreenResolution'].str.split('x',n=1,expand=True)
df['X_res'] = new[0].str[-4:].astype('int')
df['Y_res'] = new[1].astype('int')


In [ ]:
df['ppi'] = (((df['X_res']**2) + (df['Y_res']**2))**0.5/df['Inches']).astype('float')

In [ ]:
df.info()

In [ ]:
df.sample(5)

In [ ]:
# Select only numeric columns
numerical_columns = df.select_dtypes(exclude=['object'])

# Calculate correlation
numeric_columns_correlation = numerical_columns.corr()['Price']

print(numeric_columns_correlation)


->now all the data from "ScreenResolution" we taken into different columns, and we can drop this column 



In [ ]:
df.drop(columns=['ScreenResolution'],inplace=True)

In [ ]:
df.info()

next column is "Cpu" in this also we have lot of data 

we have some of categories in "Cpu" lets divide them 1st

In [ ]:
df['Cpu'].value_counts()

In [ ]:
cpu_names = []  # List to store processed CPU names


for cpu in df['Cpu']:
    x =cpu.split()[0:3]
    cpu_names.append(" ".join(x))

# Creating a new column 'Cpu Name' in the DataFrame and assign the processed CPU names
df['Cpu Name'] = cpu_names


In [ ]:
df.sample(5)

from "Cpu name" column we can fetch easily the processor

so here we have mainly 5 processors

Intel Core i3,
Intel Core i5,
Intel Core i7,
AMD Processor,
Other Intel Processor

In [ ]:
def fetch_processor(text):
    if text == 'Intel Core i7' or text == 'Intel Core i5' or text == 'Intel Core i3':
        return text
    else:
        if text.split()[0] == 'Intel':
            return 'Other Intel Processor'
        else:
            return 'AMD Processor'

processor = []  # List to store processed CPU brands

# Iterate over each value in the 'Cpu Name' column
for cpu_name in df['Cpu Name']:
    processor.append(fetch_processor(cpu_name))

# Create a new column 'Cpu brand' in the DataFrame and assign the processed CPU brands
df['processor'] = processor

In [ ]:
df.sample(5)

In [ ]:
df['processor'].value_counts().plot(kind='bar')

In [ ]:
sns.boxplot(x=df['processor'],y=df['Price'])
plt.xticks(rotation='vertical')
plt.show()

so based on Processor price is varying 

here we have taken all the data from "Cpu" column we can drop these columns 

"Cpu"
"Cpu name"

In [ ]:
df.drop(columns=['Cpu','Cpu Name'],inplace=True)

In [ ]:
df.info()

next column is "Ram" , we have strong correlation between ram and price


In [ ]:
correlation = df['Ram'].corr(df['Price'])
print("Correlation between Ram and Price:", correlation)

In [ ]:
df['Ram'].value_counts().plot(kind='bar')

In [ ]:
sns.boxplot(x=df['Ram'],y=df['Price'])
plt.xticks(rotation='vertical')
plt.show()

next column memory we have more information 

differnt types of there 

1,SSD

2,HDD

3,Flash Storage

4,combination of SSD and HDD

5,hybride

etc..

In [ ]:
df['Memory'].value_counts()

In [ ]:
# Data Cleaning and Preprocessing

df['Memory'] = df['Memory'].astype(str).replace('\.0', '', regex=True)
df["Memory"] = df["Memory"].str.replace('GB', '')
df["Memory"] = df["Memory"].str.replace('TB', '000')
new = df["Memory"].str.split("+", n=1, expand=True)



In [ ]:
df["first"]= new[0]
df["first"]=df["first"].str.strip()
df["second"]= new[1]

In [ ]:
df["Layer1HDD"] = df["first"].apply(lambda x: 1 if "HDD" in x else 0)
df["Layer1SSD"] = df["first"].apply(lambda x: 1 if "SSD" in x else 0)
df["Layer1Hybrid"] = df["first"].apply(lambda x: 1 if "Hybrid" in x else 0)
df["Layer1Flash_Storage"] = df["first"].apply(lambda x: 1 if "Flash Storage" in x else 0)

In [ ]:
df["second"].fillna("0", inplace = True)

In [ ]:
df["Layer2HDD"] = df["second"].apply(lambda x: 1 if "HDD" in x else 0)
df["Layer2SSD"] = df["second"].apply(lambda x: 1 if "SSD" in x else 0)
df["Layer2Hybrid"] = df["second"].apply(lambda x: 1 if "Hybrid" in x else 0)
df["Layer2Flash_Storage"] = df["second"].apply(lambda x: 1 if "Flash Storage" in x else 0)

In [ ]:
df['first'] = df['first'].str.replace('\D', '',regex=True)
df['second'] = df['second'].str.replace('\D', '',regex=True)

In [ ]:
df["first"] = df["first"].astype(int)
df["second"] = df["second"].astype(int)

In [ ]:
selected_columns = ['Memory',"first","second","Layer1HDD","Layer1SSD","Layer1Hybrid","Layer1Flash_Storage","Layer2HDD","Layer2SSD","Layer2Hybrid","Layer2Flash_Storage"]
subset_df = df.loc[:, selected_columns]
print(subset_df.sample(5))


In [ ]:
df["HDD"]=(df["first"]*df["Layer1HDD"]+df["second"]*df["Layer2HDD"])
df["SSD"]=(df["first"]*df["Layer1SSD"]+df["second"]*df["Layer2SSD"])
df["Hybrid"]=(df["first"]*df["Layer1Hybrid"]+df["second"]*df["Layer2Hybrid"])
df["Flash_Storage"]=(df["first"]*df["Layer1Flash_Storage"]+df["second"]*df["Layer2Flash_Storage"])

In [ ]:
selected_columns = ['Memory',"HDD","SSD","Hybrid","Flash_Storage"]
subset_df = df.loc[:, selected_columns]
print(subset_df.sample(5))

above we have taken memory column and arranged HDD, SSD ,Hybrid ,Flash_Storage

we can drop these columns now 

'first',
'second', 
'Layer1HDD', 
'Layer1SSD', 
'Layer1Hybrid',
'Layer1Flash_Storage', 
'Layer2HDD',
'Layer2SSD',
'Layer2Hybrid',
'Layer2Flash_Storage'.

In [ ]:
df.drop(columns=['Memory','first', 'second', 'Layer1HDD', 'Layer1SSD', 'Layer1Hybrid',
       'Layer1Flash_Storage', 'Layer2HDD', 'Layer2SSD', 'Layer2Hybrid',
       'Layer2Flash_Storage'],inplace=True)

In [ ]:
df.sample(5)

In [ ]:
numerical_columns = df.select_dtypes(exclude=['object'])

# Calculate correlation
numeric_columns_correlation = numerical_columns.corr()['Price']

print(numeric_columns_correlation)

if you see correlation between price and the below columns 

HDD    -0.095735

SSD              0.668174

Hybrid           0.022703

Flash_Storage   -0.034721

Hybrid and Flash_Storage have weak relation and also most of the laptops dont have this memory so we can drop these 2 columns 

In [ ]:
df.drop(columns=['Hybrid','Flash_Storage'],inplace=True)

In [ ]:
df.sample(5)

next column is "GPU" , in this also we have different catogires, for simplefication what i do is like 

i am going to create one column called Gpu brand in that 

all the intel related Gpu as intel brand

all the AMD related Gpu as AMD brand

all the Nvidia related Gpu as Nvidia brand

In [ ]:
df['Gpu'].value_counts()

In [ ]:
df['Gpu_brand'] = df['Gpu'].apply(lambda x:x.split()[0])

In [ ]:
df.sample(5)

In [ ]:
#no need of Gpu column 
df.drop(columns=['Gpu'],inplace=True)

In [ ]:
df['Gpu_brand'].value_counts()

In [ ]:
df = df[df['Gpu_brand'] != 'ARM']
df['Gpu_brand'].value_counts()

In [ ]:
sns.boxplot(x=df['Gpu_brand'],y=df['Price'])
plt.xticks(rotation='vertical')
plt.show()

we can see there is relation between Gpu brand and price

next column is "Opsys" 

In [ ]:
df['OpSys'].value_counts()

we have so many categories in this column lets create one new column called "OS"

in "OS" i divided like below

Windows 10, Windows10 S, Windows 7 as Windows 

macOS, Mac OS as Mac

other os as Others/No OS/Linux

In [ ]:
def cat_os(text):
    if text == 'Windows 10' or text == 'Windows 7' or text == 'Windows 10 S':
        return 'Windows'
    elif text == 'macOS' or text == 'Mac OS X':
        return 'Mac'
    else:
        return 'Others/No OS/Linux'

os = []  # List to store processed os names

# Iterate over each value in the 'OpSys' column
for os_name in df['OpSys']:
    os.append(cat_os(os_name))

# Create a new column 'os' in the DataFrame and assign the processed os names
df['os'] = os


In [ ]:
df.sample(5)

In [ ]:
#we can drop Opsys column

df.drop(columns=['OpSys'],inplace=True)

In [ ]:
df['os'].value_counts().plot(kind='bar')

In [ ]:
sns.boxplot(x=df['os'],y=df['Price'])
plt.xticks(rotation='vertical')
plt.show()

here we can see the windows and other os laptop price are less compare to Mac

so most of people buying windows and other os 



In [ ]:
numerical_columns = df.select_dtypes(exclude=['object'])

# Calculate correlation
numeric_columns_correlation = numerical_columns.corr()['Price']

print(numeric_columns_correlation)

here we have somany features but we dont need these many feature 

so i am droping the 'Inches','X_res', 'Y_res' because we already find the PPI 

PPI and Price correlation also strong 



In [ ]:
df.drop(columns=['Inches','X_res', 'Y_res'],inplace=True)

In [ ]:
df.head(5)

Hypothesis Testing
----

In [ ]:
df.info()

Hypothesis Testing
----

In [ ]:
df['Gpu_brand'].unique()

ANOVA Test:(Gpu_brand vs Price)

In [ ]:
from scipy.stats import f_oneway

# Extract numerical variable for ANOVA test
group1_data = df[df['Gpu_brand'] == 'Intel']['Price']
group2_data = df[df['Gpu_brand'] == 'AMD']['Price']
group3_data = df[df['Gpu_brand'] == 'Nvidia']['Price']

# Perform ANOVA test
f_statistic, p_value = f_oneway(group1_data, group2_data, group3_data)

# Print the results
print("ANOVA Test:")
print("F-statistic:", f_statistic)
print("p-value:", p_value)

# Interpret the results
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis - There are significant differences between the means of the groups.")
else:
    print("Fail to reject the null hypothesis - There are no significant differences between the means of the groups.")


ANOVA Test:(OS vs Price)

In [ ]:
df['os'].unique()

In [ ]:
from scipy.stats import f_oneway

# Extract numerical variable for ANOVA test
group1_data = df[df['os'] == 'Mac']['Price']
group2_data = df[df['os'] == 'Others/No OS/Linux']['Price']
group3_data = df[df['os'] == 'Windows']['Price']

# Perform ANOVA test
f_statistic, p_value = f_oneway(group1_data, group2_data, group3_data)

# Print the results
print("ANOVA Test:")
print("F-statistic:", f_statistic)
print("p-value:", p_value)

# Interpret the results
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis - There are significant differences between the means of the groups.")
else:
    print("Fail to reject the null hypothesis - There are no significant differences between the means of the groups.")


Correlation test

In [ ]:
numerical_columns = df.select_dtypes(exclude=['object'])

# Calculate correlation
numeric_columns_correlation = numerical_columns.corr()['Price']

print(numeric_columns_correlation)

In [ ]:
correlation_matrix = numerical_columns.corr()
sns.heatmap(correlation_matrix,cmap='coolwarm',annot=True, fmt=".2f")

Final DataSet
----

In [ ]:
df.info()

Model Development:
----

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error ,r2_score

x = df.drop(columns=['Price'])
y = np.log(df['Price'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

numeric_features=['Ram', 'Weight','Touchscreen', 'Ips', 'ppi', 'HDD', 'SSD']
categorical_features=['Company', 'TypeName', 'processor', 'Gpu_brand', 'os']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Model Training
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'GradientBoostingRegressor':GradientBoostingRegressor (random_state=42)
}
mse_dict = {}
mae_dict ={}
r2_dict={}
for name, model in models.items():
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    model_pipeline.fit(x_train, y_train)
    y_pred = model_pipeline.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    R2score = r2_score(y_test,y_pred)
    
    
    mse_dict[name] = mse
    mae_dict[name] = mae
    r2_dict[name] = R2score
    print(f"{name} MSE: {mse}")
    print(f"{name} MAE: {mae}")
    print(f"{name} R2score: {R2score}")
    print("----------------")
    

here i have choosen Random Forest algorithms to predict laptop prices.

Random Forest MSE: 0.04932216134762586

Random Forest MAE: 0.16173778229225444

Random Forest R2score: 0.8585956995981013

In [ ]:
for name, model in models.items():
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    model_pipeline.fit(x_train, y_train)
    y_pred = model_pipeline.predict(x_test)
    plt.figure(figsize=(10, 6))
    # sns.scatterplot(x=y_test, y=y_pred, label='Predicted price')
    sns.regplot(x = y_test, y=y_pred, label='Predicted price')
    plt.plot(np.arange(y_test.min(), y_test.max()), np.arange(y_test.min(), y_test.max()), color='red', label='Perfect Prediction')
    plt.xlabel('Actual df')
    plt.ylabel('Predicted df')
    plt.title(f'{name}:Predicted vs. Actual df')
    plt.legend()
    plt.show()
    

In [ ]:
object_columns = df.select_dtypes(include=['object']).columns.tolist()
numerical_columns=df.select_dtypes(exclude=['object']).columns.tolist()
print("Columns with object data type:", object_columns)
print("Columns with int and float data type:", numerical_columns)

Application
----


In [ ]:
import pandas as pd
import gradio as gr
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor



X = df.drop(columns=['Price'])
y = np.log(df['Price'])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


numeric_features = ['Ram', 'Weight', 'HDD', 'SSD']
categorical_features = ['Company', 'TypeName', 'processor', 'Gpu_brand', 'os']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


model = RandomForestRegressor(random_state=42)


pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])


pipeline.fit(X_train, y_train)

def predict_price(Company, TypeName, Ram, Weight, Inches, X_res, Y_res, Touchscreen, Ips, processor, HDD, SSD, Gpu_brand, os):
    
    ppi = (X_res**2 + Y_res**2)**0.5 / Inches
     
    input_data = pd.DataFrame({
        'Company': [Company],
        'TypeName': [TypeName],
        'Ram': [Ram],
        'Weight': [Weight],
        'Touchscreen': [Touchscreen],
        'Ips': [Ips],
        'processor': [processor],
        'HDD': [HDD],
        'SSD': [SSD],
        'Gpu_brand': [Gpu_brand],
        'os': [os],
        'ppi': [ppi]  
    })

    
    predicted_log_price = pipeline.predict(input_data)[0]
    predicted_price = np.exp(predicted_log_price)
    return round(predicted_price)

company_list = df["Company"].unique().tolist()
type_name_list = df["TypeName"].unique().tolist()
Ram_list = sorted(df["Ram"].unique().tolist()) 
processor_list = df["processor"].unique().tolist()
HDD_list = sorted(df["HDD"].unique().tolist())
SSD_list = sorted(df["SSD"].unique().tolist()) 
gpu_brand_list = df["Gpu_brand"].unique().tolist()
os_list = df["os"].unique().tolist()


interface = gr.Interface(
    fn=predict_price,
    inputs=[
        gr.Dropdown(choices=company_list, label="Company"),
        gr.Radio(choices=type_name_list, label="Type Name"),
        gr.Dropdown(choices=Ram_list, label="RAM (GB)"),
        gr.Number(label="Weight (kg)"),
        gr.Number(label="Screen Size (inches)"),
        gr.Number(label="X resolution"),
        gr.Number(label="Y resolution"),
        gr.Checkbox(label="Touchscreen"),
        gr.Checkbox(label="IPS Display"),
        gr.Radio(choices=processor_list, label="Processor"),
        gr.Dropdown(choices=HDD_list, label="HDD Storage (GB)"),
        gr.Dropdown(choices=SSD_list, label="SSD Storage (GB)"),
        gr.Radio(choices=gpu_brand_list, label="GPU Brand (if applicable)"),
        gr.Radio(choices=os_list, label="Operating System"),
    ],
    outputs=[gr.Textbox(label="Estimated Price")],
    title="Laptop Price Prediction App",
    description="Get an estimated price for your desired laptop based on its specifications.",
    allow_flagging=False  # Allow for feedback and model improvement
)


interface.launch(share=True)


Questions to Explore:
----

1,Which features have the most significant impact on laptop prices?

below are the most significant impact on laptop prices

1,Ram (0.683342): This suggests that as the RAM (memory) capacity increases, the price of the laptop tends to increase as well.

2,SSD (0.668020): This indicates that laptops with higher SSD storage capacities tend to have higher prices.

3,ppi (0.467391): Laptops with higher pixel densities may be associated with higher prices, possibly due to better display quality.

In [ ]:
numerical_columns = df.select_dtypes(exclude=['object'])

# Calculate correlation
numeric_columns_correlation = numerical_columns.corr()['Price']

print(numeric_columns_correlation)

2,Can the model accurately predict the prices of laptops from lesser-known brands?

->here in this dataset we dont have more data about lesser-known brands so thats why i didnt replace outliers because if i replace those machine will be strict to prdict only most-know or which brands have more data in dataset 

->now my modal will predict most near to Prices of all types of brands 

->if we have more data we can Divide our dataset into subsets based on these categories. 

->then if we can train on that data we will get most accurate results

3,Does the brand of the laptop significantly influence its price?

yes,brand of the laptop significantly influence its price

brands like HP,asus,dell,lenovo are midrange brand value 

but if you see Apple,MSI,microsoft have above midrange brand value 

small brands have price less only 

In [ ]:
sns.barplot(x=df['Company'],y=df['Price'],palette="colorblind")
plt.xticks(rotation='vertical')
plt.show()

ERROR:    Exception in ASGI application
  + Exception Group Traceback (most recent call last):
  |   File "c:\Users\shaik\AppData\Local\Programs\Python\Python312\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 408, in run_asgi
  |     result = await app(  # type: ignore[func-returns-value]
  |              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "c:\Users\shaik\AppData\Local\Programs\Python\Python312\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 69, in __call__
  |     return await self.app(scope, receive, send)
  |            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "c:\Users\shaik\AppData\Local\Programs\Python\Python312\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
  |     await super().__call__(scope, receive, send)
  |   File "c:\Users\shaik\AppData\Local\Programs\Python\Python312\Lib\site-packages\starlette\applications.py", line 123, in __call__
  |     await self.middleware_stack(scope, receive, send)
  |  

4,How well does the model perform on laptops with high-end specifications compared to budget laptops?

->here in this dataset we dont have more data about laptops with high-end specifications,so thats why i didnt replace outliers because if i replace those, machine will be strict to prdict only budget laptops 

->now my modal will predict most near to Prices of all types of brands 

->if we have more data about laptops with high-end specifications then we can divided dataset into subsets based on these categories. 

->then if we can train on that data then our model will perform on laptops with high-end specifications


5,What are the limitations and challenges in predicting laptop prices accurately?

limitations:

->Limited availability and quality of data ,if we have more data we can train model well

->these market have everyday new things into picture like new technology,high-GPU,Display updatation so its difficult to train model 
we have to take care of all these things 

challenges:

->Laptop prices are influenced by a wide range of factors, including specifications (e.g., processor type, RAM size, storage capacity), brand reputation, market demand, technological advancements, and economic factors. Capturing all these factors accurately in a predictive model can be big challenge

->Outliers the data, such as pricing errors, promotional discounts, or limited-time offers, can distort the predictive model's performance and lead to inaccurate predictions if not handled properly.




6,How does the model perform when predicting the prices of newly released laptops not present in the training dataset?

->if Newly released laptops have the same futures as dataset then it will perform like normal only 

->if Newly released laptops may have unique features or specifications that are not present in the training dataset. As a result, the model may struggle to find comparable products to base its predictions on, leading to less reliable estimates.

->solution is like continuously update the model with new data as it becomes available

